# Notebook for performing forecast interactively

In [ ]:
from pathlib import Path
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import datetime
import re
import sys
import copy
import os
import shutil
import json
import seaborn as sns
import itertools
from scipy import integrate, interpolate
from scipy import stats
import pickle

#from seyfert.batch import submit_utils as su
from seyfert import plotting
from seyfert.utils import *
from seyfert.config import *
from seyfert.numeric import general as nug

## Logging configuration

Put verbose to True to have all logging info

In [ ]:
import logging

verbose = False
logfile = None

logger = logging.getLogger()

if verbose:
    general_utils.configure_logger(logger, logfile=logfile)

## CMD Line arguments

In [ ]:
do_compute_derivatives = True

arguments = {
    "forecast_config": None,
    "run_id": None,
    "input_data_dir": Path.home() / "spectrophoto/input_data",
    "powerspectrum_dir": None,
    "angular_dir": None,
    "derivative_dir": None,
    "fisher_dir": None,
    "powerspectrum_config": filesystem_utils.config_files_dir() / 'power_spectrum_config.json',
    "angular_config":       filesystem_utils.config_files_dir() / 'angular_config.json',
    "derivative_config":    filesystem_utils.config_files_dir() / 'derivative_config.json',
    "fisher_config":        filesystem_utils.config_files_dir() / 'fisher_config.json',
    "results_config":       filesystem_utils.config_files_dir() / 'results_config.json',
    "execution": "no",
    "n_cores_per_job": 1,
    "queue": None,
    "memory": None,
    "verbose": False,
    "test": True,
    "doctor": False,
    "ignore_errfiles": False,
    "powerspectrum": False,
    "angular": True,
    "derivative": True,
    "fisher": True,
    "ini_file": None,
}

In [ ]:
arguments["forecast_config"]   = "/Users/lucapaganin/spectrophoto/config_files/1.3.5/optimistic/optm_ell_log_10_3000_4_sp_bins_1.3.5_2021-04-01T11-40-30.xml"
arguments["powerspectrum_dir"] = "/Users/lucapaganin/spectrophoto/powerspectra/ISTF_WL_Flat_powerspectra_1.2.1_2020-11-13T13-52-11/"

arguments["execution"] = "interactive"
arguments["test"] = False

## Setup

In [ ]:
from seyfert.batch import submit_utils as su

In [ ]:
runner = su.ForecastRunner(cmd_line_args=arguments)
runner.buildTasks()
runner.prepareWorkspace()
runner.buildJobSubmitters()

In [ ]:
#runner.run()
#rundir = runner.rundir

## Delta Cls

In [ ]:
from seyfert.fisher.delta_cl import DeltaClCollection

delta_cls = DeltaClCollection(fcfg=fcfg, fisher_cfg=fisher_config)
delta_cls.loadFiducialCls(fid_cls)
delta_cls.evaluateSingleBlocks()
delta_cls.buildXCBlocks()

if not (workdir / "delta_cls.h5").exists():
    delta_cls.saveToHDF5(workdir / "delta_cls.h5")

## Fisher

In [ ]:
from seyfert.fisher import fisher_computation as fcomp
from seyfert.fisher.fisher_matrix import FisherMatrix
from seyfert.base_structs.cl_dict import ClDict

In [ ]:
logfile = None

logger = logging.getLogger()
general_utils.configure_logger(logger, logfile=logfile)

In [ ]:
flex_ders_dict = {
    dvar: ClDict({key: dcl.dc_lij for key, dcl in dcoll.dcl_dict.items()})
    for dvar, dcoll in ders_dict.items()
}

In [ ]:
ti = time.time()
print("computing fisher matrices")

probes = ["WL", "GCph", "GCsp"]

fish_outdir = workdir / "fishers"
fish_outdir.mkdir(exist_ok=True)

fmats = fcomp.compute_all_fishers(probes, params, flex_ders_dict, delta_cls, fish_outdir, silent=True, save_f_ell_auto=False)

fmats["GCsp(Pk)"] = FisherMatrix.fromHDF5("../fisher_compute/data/ist_fishers/EuclidISTF_GCsp_w0wa_flat_optimistic.hdf5")

if "GCsp" in fmats:
    fmats['GCsp(Cl)'] = fmats['GCsp']
    fmats['GCsp(Cl)'].name = 'GCsp(Cl)'
    del fmats['GCsp']

tf = time.time()
print(f"Elapsed time {formatters.string_time_format(tf-ti)}")

### Fishers of autocorrelations to be added

In [ ]:
add_autos_contribs = True

#### WL

In [ ]:
if add_autos_contribs:
    fisher_WL_3000_5000 = FisherMatrix.fromHDF5("autocorr_fishers/optimistic/fisher_WL_ell_3000_5000.hdf5")

    for key in fmats:
        if "WL" in key:
            fmats[key].matrix = fmats[key].matrix.add(fisher_WL_3000_5000.matrix, fill_value=0)
        fmats[key].evaluateInverse()

Writing fisher matrices to file